#### Imports

In [1]:
import os
import re
import skrf as rf
import pandas as pd

#### Paths

In [2]:
script_directory = os.getcwd()#obtem diretório atual
s2ps_path = os.path.join(script_directory,'..','..','oil_data','oil_data') #cria string caminho para todos os .s2p

#### Regex = hell

In [3]:
regex = r'SAW_06_02_2024_OIL_TRAFO_44_(\d+)\.S2P'

In [4]:
def read_s2p_data(regex, path):
    dfs = []#lista para ocncatenar todos os dicts de cada medição 
    for filename in os.listdir(path):
        match = re.match(regex, filename)
        if match:
            #obtem informação do nome do arquivo
            oil_name = "TRAFO 44"
            measurement_number = int(match.group(1))
            
            #carrega arquivo
            path_to_oil = os.path.join(path, filename)
            s2p_file = rf.Network(path_to_oil)
            
            #extrai frequência e parâmetros S do objeto Network
            frequency = s2p_file.f
            s_parameters = s2p_file.s_db
            
            #Dataframe com informações do objeto Network
            individual_oil_dict = {
                'OIL NAME': oil_name,
                'MEASUREMENT NUMBER': measurement_number,
                'FREQUENCY (MHZ)':frequency/1000000,
                'S11 (db)' : s_parameters[:,0,0],
                'S21 (db)' : s_parameters[:,1,0],
                'S12 (db)' : s_parameters[:,0,1],
                'S22 (db)' : s_parameters[:,1,1],
            }
            df = pd.DataFrame.from_dict(individual_oil_dict)
            dfs.append(df)
            
    dataframe = pd.concat(dfs, ignore_index=True)
    return dataframe

#### Read Dataframes

In [5]:
df_mineral = read_s2p_data(regex, s2ps_path)

In [6]:
df_mineral.info

<bound method DataFrame.info of        OIL NAME  MEASUREMENT NUMBER  FREQUENCY (MHZ)  S11 (db)  S21 (db)  \
0      TRAFO 44                   1       110.000000 -0.932214 -78.45488   
1      TRAFO 44                   1       110.002000 -0.933999 -78.86480   
2      TRAFO 44                   1       110.004000 -0.933206 -77.32688   
3      TRAFO 44                   1       110.006001 -0.934983 -78.64076   
4      TRAFO 44                   1       110.008001 -0.934740 -80.71636   
...         ...                 ...              ...       ...       ...   
29995  TRAFO 44                   3       129.991999 -1.267004 -69.87478   
29996  TRAFO 44                   3       129.993999 -1.268708 -69.31310   
29997  TRAFO 44                   3       129.996000 -1.269654 -69.27567   
29998  TRAFO 44                   3       129.998000 -1.269770 -67.58872   
29999  TRAFO 44                   3       130.000000 -1.268233 -67.16007   

       S12 (db)  S22 (db)  
0     -77.12537 -1.131002  

In [7]:
#assegura somente óleo de girassol lido
for name in df_mineral["OIL NAME"]:
    if name != "TRAFO 44":
        print("Diferente de TRAFO 44")

In [8]:
#cria .csv caso precise no futuro.
path_save_csv = os.path.join(script_directory,'..','..', 'oil_data','oil_data','CSVs')
if not os.path.exists(path_save_csv):
   os.makedirs(path_save_csv)
df_mineral.to_csv(path_save_csv + '/TRAFO_44_measurements.csv')

In [9]:
def average_measurements(dataframe):
    #agrupa valores por frequência e calcula média do que interessa
    return dataframe.groupby('FREQUENCY (MHZ)', as_index=False)['S21 (db)'].mean()

In [10]:
df_TRAFO_44_s21 = average_measurements(df_mineral)

In [11]:
df_TRAFO_44_s21.info


<bound method DataFrame.info of       FREQUENCY (MHZ)   S21 (db)
0          110.000000 -82.068840
1          110.002000 -76.808343
2          110.004000 -78.968813
3          110.006001 -81.446937
4          110.008001 -79.061737
...               ...        ...
9995       129.991999 -69.576893
9996       129.993999 -69.541777
9997       129.996000 -69.118643
9998       129.998000 -67.934313
9999       130.000000 -67.406780

[10000 rows x 2 columns]>

In [12]:
#cria .csv caso precise no futuro.
path_save_csv_processed = os.path.join(script_directory,'..','..', 'oil_data','oil_data','CSVs','processed')
if not os.path.exists(path_save_csv_processed):
   os.makedirs(path_save_csv_processed)
df_TRAFO_44_s21.to_csv(path_save_csv_processed + '/TRAFO_44_measurements_processed.csv')